# Deep Learning with python Keras
## Chapter 7: Advanced deep learning best practices
### 7.1 Going beyong the sequential model: the Keras functional API

In this tutorial, we only concentrate on the type (or the construction) of different model. For example, the multi-input, multi-output model and so on.

#### 7.1.1 Introduction to the functional API

In [16]:
from keras import Input, layers, losses, optimizers, metrics
from keras.models import Sequential, Model
import numpy as np
from keras.utils import plot_model
import matplotlib.pyplot as plt
import os

In [18]:
# set the results file path
sectionnum = '7_1'
cur_work_path = os.getcwd()
res_path = '{}/res_c{}'.format(cur_work_path,sectionnum)
if not os.path.exists(res_path):
    os.mkdir(res_path)

In [21]:
# A Sequential model
seq_model = Sequential()
seq_model.add(layers.Dense(64,activation='relu',input_shape=(64,)))
seq_model.add(layers.Dense(32,activation='relu'))
seq_model.add(layers.Dense(10,activation='softmax'))
seq_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# A functional equivalent
input_tensor = Input(shape=(64,),name='input_tensor')
x = layers.Dense(64, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

fun_model = Model(input_tensor, output_tensor)
fun_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# let compare the two models
print('the sequential model:', seq_model.summary())
print('the functional model:', fun_model.summary())

plot_model(model=seq_model,
          to_file='{}/simple_seq_model.png'.format(res_path),
          show_shapes=True)
plot_model(model=fun_model,
          to_file='{}/simple_fun_model.png'.format(res_path),
          show_shapes=True)

# training models
x_train = np.random.random((1000,64))
y_train = np.random.random((1000,10))

fun_model.fit(x_train, y_train, epochs=10, batch_size=128)
score = fun_model.evaluate(x_train, y_train)
print(score)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_51 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_52 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_53 (Dense)             (None, 10)                330       
Total params: 6,570
Trainable params: 6,570
Non-trainable params: 0
_________________________________________________________________
the sequential model: None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_tensor (InputLayer)    (None, 64)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 64)                4160      
_______________________________________________

#### 7.1.2 Multi-input models

In [26]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocanulary_size = 500

# text input
text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

# question input
question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(32, question_vocabulary_size)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# the we concantenate the two 
concatenated = layers.concatenate([encoded_text,encoded_question], axis=-1)
answer = layers.Dense(answer_vocanulary_size, activation='softmax')(concatenated)

multi_input_model = Model([text_input, question_input], answer)
multi_input_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['acc'])

print('the multi-input model:', multi_input_model.summary())

plot_model(model=multi_input_model,
          to_file='{}/multi_input_model.png'.format(res_path),
          show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 10000)  640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 10000)  320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_3 (LS

In [33]:
# Let's generate some dummy numpu data
num_samples = 10
max_len = 50
# notes: the second param 32, should not be larger than 64, which is the first param in 
# layers.Embedding(64, text_vocabulary_size)(text_input)
text = np.random.randint(1, 32, size=(num_samples, max_len))
question = np.random.randint(1, 12, size=(num_samples, max_len))
# answers are one-hot encoded
answers = np.random.randint(0,1,size=(num_samples,answer_vocanulary_size))
# Fitting
multi_input_model.fit([text,question], answers, epochs=10, batch_size=128)
print('Train it again...')
multi_input_model.fit({'text': text, 'question': question},answers,epochs=5, batch_size=128)

Epoch 1/10
10/10 [==============================] - 1s 69ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 2/10
10/10 [==============================] - 1s 66ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 3/10
10/10 [==============================] - 1s 66ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 4/10
10/10 [==============================] - 1s 67ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 5/10
10/10 [==============================] - 1s 73ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 6/10
10/10 [==============================] - 1s 80ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 7/10
10/10 [==============================] - 1s 65ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 8/10
10/10 [==============================] - 1s 65ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 9/10
10/10 [==============================] - 1s 78ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 10/10
10/10 [==============================] - 1s 68ms/step - loss: 0.0000e+

#### 7.1.3 Multi-output models

In [35]:
vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,),dtype='int32',name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dense(128, activation='relu')(x)

# the output layers
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

multi_output_model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

multi_output_model.compile(optimizer='rmsprop',
                         loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
                         loss_weights = [0.25, 1., 10.])
print('the multi-ouput model:', multi_input_model.summary())

plot_model(model=multi_output_model,
          to_file='{}/multi_ouput_model.png'.format(res_path),
          show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 10000)  640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 10000)  320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_3 (LS

#### 7.1.4 Directed acylic graphs of layers

In [40]:
# the input is a 4D tensor
x = Input(shape=(None,640,480), dtype='float32',name='input_x')
# we apply some transfromation to 'x'
y = layers.Conv2D(128, 3, activation='relu')(x)
y = layers.Conv2D(128, 3, activation='relu')(y)
y = layers.Conv2D(128, 3, activation='relu')(y)

# we add the original 'x' back to the output features
y = layers.add([y,x])

print('the direct acylic model:', multi_input_model.summary())

plot_model(model=multi_output_model,
          to_file='{}/direct_acylic_model.png'.format(res_path),
          show_shapes=True)

ValueError: Operands could not be broadcast together with shapes (None, 634, 128) (None, 640, 480)

#### 7.1.5 Layer weight sharing